In [69]:
import pandas as pd
from datetime import datetime
import re
import os
import numpy as np

In [56]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', ' Diciembre']
hoy = datetime.now()

mes_actual_int = '0' + str(hoy.month)
mes_actual_str = meses[hoy.month - 1]
mes_anterior_int = '0' + str(hoy.month - 1)
mes_anterior_str = meses[hoy.month - 2]

In [57]:
ruta_afiliaciones = f'C:/Users/BBARTOLOMEE/OneDrive - Entel Peru S.A/Afiliaciones/2025/{mes_anterior_int}. {mes_anterior_str}'

In [58]:
ruta_afiliaciones

'C:/Users/BBARTOLOMEE/OneDrive - Entel Peru S.A/Afiliaciones/2025/06. Junio'

In [59]:
#Buscar archivo afiliaciones para obtener su nombre actual
def buscar_nombre_afiliaciones(ruta_afiliaciones):
        for archivo in os.listdir(ruta_afiliaciones):
                if archivo.lower().startswith('afiliaciones'):
                        nombre = archivo
                        return nombre

In [60]:
nombre_afiliaciones = buscar_nombre_afiliaciones(ruta_afiliaciones)
print(nombre_afiliaciones)

Afiliaciones_Junio_25.xlsx


In [61]:
ruta_afiliaciones_completa = ruta_afiliaciones + '/' + nombre_afiliaciones
df_afiliaciones = pd.read_excel(ruta_afiliaciones_completa, sheet_name= f'Afiliaciones {mes_anterior_str} 25')

In [ ]:
df_afiliaciones

<h4>CHECK PEDIDO<h4>

In [63]:
ruta_pedido = f'C:/Users/BBARTOLOMEE/OneDrive - Entel Peru S.A/Afiliaciones/2025/{mes_anterior_int}. {mes_anterior_str}/Inputs'

In [64]:
def ultimo_pedido(ruta_pedido):
    prefijos_dict = {}

    for archivo in os.listdir(ruta_pedido):
        if archivo.lower().startswith('pedido'):
            prefijo = archivo[10:12]
            try:
                clave = int(prefijo)
                if clave not in prefijos_dict:
                    prefijos_dict[clave] = []
                prefijos_dict[clave].append(archivo)
            except ValueError:
                print(f"'{archivo}' no tiene un prefijo numérico válido.")

    if not prefijos_dict:
        print("No se encontraron archivos con prefijos numéricos válidos.")
        return None

    clave_max = max(prefijos_dict)
    return prefijos_dict[clave_max][0]

In [65]:
primer_pedido = ultimo_pedido(ruta_pedido)
ruta_pedido_completa = ruta_pedido + "/" + primer_pedido

In [ ]:
df_pedido_original = pd.read_excel(ruta_pedido_completa)

In [ ]:
df_pedido_afiliaciones = df_pedido_original.drop_duplicates(subset='RUCCOMPANIA')
df_pedido_afiliaciones

In [ ]:
df_afiliaciones_col1 = pd.merge(df_afiliaciones, df_pedido_afiliaciones[['RUCCOMPANIA','LARGO']], on='RUCCOMPANIA', how='left')


In [ ]:
df_afiliaciones_col1

In [ ]:
df_afiliaciones_col1['OPS VÍA E-MAIL 02.05'] = 'NO'

condicion = df_afiliaciones_col1['LARGO'].notna() & (df_afiliaciones_col1['LARGO'] != '') & (df_afiliaciones_col1['LARGO'].astype(str).str.strip() != '')

df_afiliaciones_col1.loc[condicion, 'OPS VÍA E-MAIL 02.05'] = 'SI'

df_afiliaciones_col1 = df_afiliaciones_col1.drop('LARGO',axis = 'columns')

<h4>CT<h4>

In [67]:
def excel_ct(ruta_pedido, input):
    nombre = ''
    for archivo in os.listdir(ruta_pedido):
        if archivo.lower().startswith(input):
            nombre=archivo
            break
    return nombre

In [68]:
primer_ct = excel_ct(ruta_pedido,'ct')

In [ ]:
df_ct = pd.read_excel(ruta_pedido + '/' + primer_ct)

In [ ]:
df_ct.drop_duplicates(subset = "RUCCOMPANIA", inplace=True)
df_ct.reset_index(inplace = True, drop = True)
df_ct

In [ ]:
df_afiliaciones_col2 = pd.merge(df_afiliaciones_col1, df_ct[['RUCCOMPANIA','LARGO']], on='RUCCOMPANIA', how='left')

In [ ]:
len(df_afiliaciones_col2)

In [ ]:
df_afiliaciones_col2['OPS VÍA VOZ 02.05'] = 'NO'

condicion = df_afiliaciones_col2['LARGO'].notna() & (df_afiliaciones_col2['LARGO'] != '') & (df_afiliaciones_col2['LARGO'].astype(str).str.strip() != '')

df_afiliaciones_col2.loc[condicion, 'OPS VÍA VOZ 02.05'] = 'SI'

df_afiliaciones_col2 = df_afiliaciones_col2.drop('LARGO',axis = 'columns')

In [ ]:
condicion_2 = [(df_afiliaciones_col2['OPS VÍA E-MAIL 02.05'] == 'SI') & (df_afiliaciones_col2['OPS VÍA VOZ 02.05'] == 'SI'),
               (df_afiliaciones_col2['OPS VÍA E-MAIL 02.05'] == 'SI') & (df_afiliaciones_col2['OPS VÍA VOZ 02.05'] == 'NO'),
               (df_afiliaciones_col2['OPS VÍA E-MAIL 02.05'] == 'NO') & (df_afiliaciones_col2['OPS VÍA VOZ 02.05'] == 'SI'),
               (df_afiliaciones_col2['OPS VÍA E-MAIL 02.05'] == 'NO') & (df_afiliaciones_col2['OPS VÍA VOZ 02.05'] == 'NO')]

valores = ['AFILIADO COMPLETO','AFILIADO SOLO MAIL','AFILIADO SOLO VOZ','NO AFILIADO']

df_afiliaciones_col2['STATUS AFILIACION 02.05'] = np.select(condicion_2, valores, default='')

In [ ]:
df_afiliaciones_col2

In [ ]:
#Quitar numeros no validos de pedido

df_pedido_telf_real = df_pedido_afiliaciones[df_pedido_afiliaciones['LARGO'] != 1]

In [ ]:
df_afiliaciones_col3 = pd.merge(df_afiliaciones_col2, df_pedido_telf_real[['RUCCOMPANIA','LARGO']], on='RUCCOMPANIA', how='left')

In [ ]:
df_afiliaciones_col3['Pedido con Cel'] = 'NO'

condicion = df_afiliaciones_col3['LARGO'].notna()

df_afiliaciones_col3.loc[condicion, 'Pedido con Cel'] = 'SI'

df_afiliaciones_col3 = df_afiliaciones_col3.drop('LARGO',axis = 'columns')

In [ ]:
df_afiliaciones_col3

In [ ]:
condicion_3 = [(df_afiliaciones_col3['STATUS AFILIACION 02.05'] == 'AFILIADO COMPLETO') & (df_afiliaciones_col3['Pedido con Cel'] == 'SI'),
               (df_afiliaciones_col3['STATUS AFILIACION 02.05'] == 'AFILIADO SOLO MAIL') & (df_afiliaciones_col3['Pedido con Cel'] == 'SI'),
               (df_afiliaciones_col3['STATUS AFILIACION 02.05'] == 'AFILIADO COMPLETO') & (df_afiliaciones_col3['Pedido con Cel'] == 'NO'),
               (df_afiliaciones_col3['STATUS AFILIACION 02.05'] == 'AFILIADO SOLO MAIL') & (df_afiliaciones_col3['Pedido con Cel'] == 'NO'),
               (df_afiliaciones_col3['STATUS AFILIACION 02.05'] == 'AFILIADO SOLO VOZ') & (df_afiliaciones_col3['Pedido con Cel'] == 'NO'),
               (df_afiliaciones_col3['STATUS AFILIACION 02.05'] == 'NO AFILIADO') & (df_afiliaciones_col3['Pedido con Cel'] == 'NO'),]

valores_2 = ['AFILIADO COMPLETO','AFILIADO COMPLETO','AFILIADO COMPLETO','AFILIADO SOLO MAIL','AFILIADO SOLO VOZ','NO AFILIADO']

df_afiliaciones_col3['STATUS AFILIACION FINAL 02.05'] = np.select(condicion_3, valores_2, default='')

In [ ]:
df_afiliaciones_col3.head(5)

<h4>Subimos rol adm<h4>

In [ ]:
primer_rol = excel_ct(ruta_pedido,'rol')
df_rol = pd.read_excel(ruta_pedido + '/' + primer_rol)

In [ ]:
#Prueba
#df_rol = pd.read_excel(r'C:\Users\BBARTOLOMEE\OneDrive - Entel Peru S.A\Afiliaciones\2025\04. Abril\Inputs\Rol Online 26.06.xlsx')

In [ ]:
df_rol

In [ ]:
df_rol['51NUM'] = '51' + df_rol['LINEA'].fillna(0).astype(int).astype(str)
df_rol['LLAVE'] = df_rol['RUC'].fillna(0).astype(int).astype(str) + df_rol['51NUM'].astype(str)

In [ ]:
df_rol

<H4>SUSCRIPTORES<H4>

In [7]:
ruta_suscriptores = f'C:/Users/BBARTOLOMEE/OneDrive - Entel Peru S.A/Afiliaciones/2025/{mes_anterior_int}. {mes_anterior_str}/Suscriptores'

In [8]:
def ultimo_suscriptores(ruta_carpeta):
    prefijos_dict = {}

    for archivo in os.listdir(ruta_carpeta):
        if archivo.lower().endswith('.xlsx'):
            prefijo = archivo[16:18]
            try:
                clave = int(prefijo)
                if clave not in prefijos_dict:  
                    prefijos_dict[clave] = []
                prefijos_dict[clave].append(archivo)
            except ValueError:
                print(f"'{archivo}' no tiene un prefijo numérico válido.")

    if not prefijos_dict:
        print("No se encontraron archivos con prefijos numéricos válidos.")
        return None

    clave_max = max(prefijos_dict)
    return prefijos_dict[clave_max][0]

In [9]:
ultimo_suscriptores = ultimo_suscriptores(ruta_suscriptores)

In [10]:
ultimo_suscriptores

'Suscriptores.01.07.xlsx'

In [11]:
suscriptores = pd.read_excel(ruta_suscriptores + "/" + ultimo_suscriptores, sheet_name=None)

In [13]:
df_suscriptores_smart = suscriptores['Smart']
df_suscriptores_no_smart = suscriptores['No Smart']

In [ ]:
len(df_rol)

In [ ]:
df_rol_1 = pd.merge(df_rol,df_suscriptores_smart[['LLAVE','LONGITUDCOMPANIA']], on= 'LLAVE', how = 'left')

In [ ]:
df_cruzados_llave = df_rol_1[df_rol_1['LONGITUDCOMPANIA'].notna()].copy()
df_no_cruzados_llave = df_rol_1[df_rol_1['LONGITUDCOMPANIA'].isna()].copy()

In [ ]:
df_no_cruzados_llave_NoSmart = pd.merge(df_no_cruzados_llave, df_suscriptores_no_smart[['LLAVE','LONGITUDCOMPANIA']],on = 'LLAVE', how = 'left')

In [ ]:
df_rol_2 = pd.concat([df_cruzados_llave, df_no_cruzados_llave_NoSmart], ignore_index=True)

In [ ]:
len(df_rol_2)

In [ ]:
df_rol_2['ETIQUETA'] = df_rol_2['LONGITUDCOMPANIA'].combine_first(df_rol_2['LONGITUDCOMPANIA_x']).combine_first(df_rol_2['LONGITUDCOMPANIA_y'])

In [ ]:
df_rol_2.drop(columns=['LONGITUDCOMPANIA_x','LONGITUDCOMPANIA_y','LONGITUDCOMPANIA'], inplace=True)

In [ ]:
condicion_5 = df_rol_2['ETIQUETA'].notna() & (df_rol_2['ETIQUETA'] != '')
condicion_6 = df_rol_2['ETIQUETA'].isna()

df_rol_2.loc[condicion_5, 'ETIQUETA'] = 'SI'
df_rol_2.loc[condicion_6, 'ETIQUETA'] = 'NO'

In [ ]:
#Filtrar ROL y Suscriptores y quedarnos con RUCS y otro campo más para cruzarlo
df_rol_3 = df_rol_2[(df_rol_2['ROL'] == 'ADMINISTRADOR') & (df_rol_2['ETIQUETA'] == 'SI')]

In [ ]:
df_rol_3_TD1 = df_rol_3[['RUC', 'ETIQUETA']]

In [ ]:
df_rol_3_TD1.reset_index(drop = True, inplace = True)

In [ ]:
df_rol_3_TD1.rename(columns = {'RUC' : 'RUCCOMPANIA'}, inplace = True)

In [ ]:
df_rol_3_TD1.drop_duplicates(subset = 'RUCCOMPANIA',inplace= True)
df_rol_3_TD1

In [ ]:
df_afiliaciones_col4 = pd.merge(df_afiliaciones_col3, df_rol_3_TD1, on = 'RUCCOMPANIA', how = 'left' )
df_afiliaciones_col4.rename(columns = {'ETIQUETA': 'STATUS ONLINE 02.05'}, inplace = True)
df_afiliaciones_col4['STATUS ONLINE 02.05'].fillna('NO', inplace = True)

In [ ]:
df_afiliaciones_col4

In [ ]:
df_afiliaciones_cruce = df_afiliaciones_col4.copy()

In [ ]:
df_afiliaciones_cruce.drop_duplicates(subset='RUCCOMPANIA',inplace = True)

In [ ]:
#cruzar con afiliaciones
df_rol_2.rename(columns = {'RUC':'RUCCOMPANIA'}, inplace = True)
df_rol_4 = pd.merge(df_rol_2, df_afiliaciones_cruce[['RUCCOMPANIA','MODELO DE ATENCIÓN MES']], on='RUCCOMPANIA', how= 'left')

In [ ]:
df_pedido = df_pedido_original[df_pedido_original['LARGO'] != 1]
df_pedido = df_pedido[['LLAVE','RUCCOMPANIA']]
df_pedido = df_pedido.drop_duplicates(subset = 'LLAVE')

df_rol_5 = pd.merge(df_rol_4, df_pedido, on = 'LLAVE', how='left')

In [ ]:
df_rol_5['CA'] = 'NO'

condicion = df_rol_5['RUCCOMPANIA_y'].notna()

df_rol_5.loc[condicion, 'CA'] = 'SI'

df_rol_5 = df_rol_5.drop('RUCCOMPANIA_y',axis = 'columns')

In [ ]:
df_rol_5

In [ ]:
#IVR
#ruta_IVR = f'C:/Users/BBARTOLOMEE/OneDrive - Entel Peru S.A/Afiliaciones/2025/IVR/{mes_anterior_str}.xlsx'

In [ ]:
#df_ivr = pd.read_excel(ruta_IVR)

In [ ]:
#df_ivr['51CEL'] = '51' + df_ivr['TELEFONO'].astype(str)
#df_ivr['LLAVE'] = df_ivr['SWRUC'].astype(str) + df_ivr['51CEL'].astype(str)

In [ ]:
#df_ivr

In [ ]:
#df_ivr.drop_duplicates(subset = 'LLAVE',inplace = True)
#df_ivr_1 = df_ivr[['LLAVE', '51CEL']]

In [ ]:
#df_rol_6 = pd.merge(df_rol_5, df_ivr_1, on = 'LLAVE', how='left')

In [ ]:
#df_rol_6.info()

In [ ]:
#df_rol_6['IVR'] = 'NO'

#condicion_7 = df_rol_6['51CEL'].notna()

#df_rol_6.loc[condicion_7, 'IVR'] = 'SI'

#df_rol_6.drop('51CEL',axis = 'columns', inplace= True)

In [ ]:
#df_rol_6

In [ ]:
#condicion_8 = [(df_rol_6['PEDIDO'] == 'NO') & (df_rol_6['IVR'] == 'NO'),]

#valores_2 = ['NO']

#df_rol_6['CA'] = np.select(condicion_8, valores_2, default='SI')

In [ ]:
#df_rol_6

In [ ]:
df_rol_5.info()

In [ ]:
conteo_productos = df_rol_5['CA'].value_counts()

print("Conteo de cada producto:")
conteo_productos

In [ ]:
# filtrar los que tengan un modelo atencion y suscriptores SI
df_rol_base_nueva = df_rol_5[(df_rol_5['ETIQUETA'] == 'SI') & (df_rol_5['MODELO DE ATENCIÓN MES'] != 'Sin modelo de atencion') &(df_rol_5['MODELO DE ATENCIÓN MES'].notna())]

# filtramos los que sean solo administradores 
df_rol_adm = df_rol_base_nueva[df_rol_base_nueva['ROL'] == 'ADMINISTRADOR']

In [ ]:
conteo_etiqueta = df_rol_base_nueva['CA'].value_counts()

print(conteo_etiqueta)

In [ ]:
df_rol_base_nueva

In [ ]:
df_rol_adm

In [ ]:
#df_rol_base_nueva['ROL'].unique().tolist()

In [ ]:
df_rol_adm.drop_duplicates(subset='LLAVE',inplace = True)

In [ ]:
df_rol_ca = df_rol_base_nueva[df_rol_base_nueva['CA'] == 'SI']

In [ ]:
df_rol_ca

In [ ]:
df_rol_adm['ROLX'] = 'ROL ADM'

In [ ]:
df_rol_adm

In [ ]:
df_rol_adm_cruce = df_rol_adm.drop_duplicates(subset = 'LLAVE')

In [ ]:
df_rol_ca_ultimo = pd.merge(df_rol_ca, df_rol_adm_cruce[['LLAVE', 'ROLX' ]], on = 'LLAVE', how = 'left')

In [ ]:
df_rol_ca_ultimo.rename(columns = {'ROLX':'ROL3'},inplace=True)

In [ ]:
conteo_etiqueta = df_rol_ca_ultimo['ROL3'].value_counts()

print(conteo_etiqueta)

In [ ]:
df_rol_final = df_rol_ca_ultimo[df_rol_ca_ultimo['ROL3'] == 'ROL ADM']

In [ ]:
df_rol_final.rename(columns = {'RUCCOMPANIA_x':'RUCCOMPANIA'}, inplace=True)

In [ ]:
df_rol_final

In [ ]:
df_rol_final.drop_duplicates(subset='RUCCOMPANIA', inplace =True)

In [ ]:
df_afiliaciones_col5 = pd.merge(df_afiliaciones_col4, df_rol_final[['RUCCOMPANIA','ROL3']], on='RUCCOMPANIA', how = 'left')

In [ ]:
df_afiliaciones_col5

In [ ]:
condicion_9 = [(df_afiliaciones_col5['STATUS ONLINE 02.05'] == 'NO') & (df_afiliaciones_col5['ROL3'].isna()),
               (df_afiliaciones_col5['STATUS ONLINE 02.05'] == 'SI') & (df_afiliaciones_col5['ROL3'] == 'ROL ADM'),
               (df_afiliaciones_col5['STATUS ONLINE 02.05'] == 'SI') & (df_afiliaciones_col5['ROL3'].isna())]

valores_2 = ['NO','AUTORIZADO','NO AUTORIZADO']

df_afiliaciones_col5['ADMINISTRADOR CA 02.05'] = np.select(condicion_9, valores_2, default='')

df_afiliaciones_col5.drop(columns=['ROL3'], inplace = True)

In [ ]:
df_afiliaciones_col5

In [ ]:
df_afiliaciones_col5.to_excel('Reporte Afiliaciones.xlsx', index = False)